In [1]:
import csv
import json

with open('/content/filtered_tables_dataset_validated.json', 'r') as f:
  data = json.load(f)





Todos: We can include comments in the table schema

In [10]:
from typing_extensions import dataclass_transform
def create_prompt(raw_user_query, specified_table, specified_table_schema):
    """
    Create a formatted prompt asking if a specified table (with schema) should be used
    for generating a SQL query based on the raw user query and candidate tables.

    Parameters:
    - raw_user_query (str): The user's raw SQL query or question.
    - specified_table_schema ( str): A list of table definitions (table name and schema)
      that could be included in the SQL query.
    - specified_table (str): The specific table definition (table name and schema) to check.

    Returns:
    - str: The formatted prompt.
    """

    # Construct the prompt message
    # prompt = (
    #     f"User is asking for a SQL query that performs the following task. UserQuery: '{raw_user_query}' \n"
    #     f"Question: Should the {specified_table} table be used to generate the SQL query that addresses the UserQuery? \n"
    # )

    single_line_sep_prompt = {
        f"{raw_user_query} [SEP] {specified_table}"
    }

    return single_line_sep_prompt

csv_rows = []

for obj in data:
    included_tables = set()
    for included in obj["llm_filtered_tables"].split(","):
        included = included.strip()
        if included not in obj["rag_filtered_tables"]:
            print("Didnt found the table in the json obj")
            exit()
        included_tables.add(included)

    not_included_tables = list(set([key for key in obj["rag_filtered_tables"]]) - included_tables)
    included_tables = list(included_tables)
    count_not_included_table_processes = 0

    for table_name in obj['rag_filtered_tables']:
        if table_name in included_tables:
            label = 1
        else:
            count_not_included_table_processes += 1
            label = 0

        # Toggle this if statement to get the imbalanced dataset
        if False and label == 0 and count_not_included_table_processes  > len(included_tables):
            pass
        else:
            specified_table_schema = obj['rag_filtered_tables'][table_name]['schema']
            specified_table_schema = specified_table_schema.replace("\n", " ")

            prompt = create_prompt(obj['user_raw_query'], table_name, specified_table_schema)
            csv_rows.append([prompt, label])

csv_file_path = "single_line_sep_prompt_imbalanaced.csv"

with open(csv_file_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Prompt", "Label"])  # Header
    writer.writerows(csv_rows)

print(f"CSV file saved at {csv_file_path}")




CSV file saved at single_line_sep_prompt_imbalanaced.csv


In [ ]:
# prompt: create a function which would add data to the csv file

import csv

def add_data_to_csv(file_path, data):
    """
    Adds a row of data to a CSV file.

    Args:
        file_path (str): The path to the CSV file.
        data (list): A list representing the row of data to add.
    """
    try:
        # Check if the file exists
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            header = next(reader, None)  # Get the header row if it exists
    except FileNotFoundError:
        header = None

    with open(file_path, 'a', newline='') as file:
        writer = csv.writer(file)

        # Write the header if the file is newly created
        if header is None:
            writer.writerow(['user_raw_query', 'table_name', 'schema', 'label'])  # Define the header row

        writer.writerow(data)


0


In [9]:
import pandas as pd

df = pd.read_csv('/content/single_line_sep_prompt_balanaced.csv')

from collections import Counter

freq = Counter(df['Label'])

freq

Counter({1: 1427, 0: 1418})